In [189]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
import sys

In [213]:
cards = list(range(1,11))+[10]*3
# moves = hit, stand(, split, double)

# for each hand, there's an expected outcome    
# if the win probability for the current hand is better then the win probability of the expected hand, stand, otherwise hit.
columns = range(1,33)
index = range(1,33)
stand_probs = pd.DataFrame(columns=columns, index=index)
def stand_prob(my_hand, dealer_hand):
    if not pd.isna(stand_probs[my_hand][dealer_hand]):
        return stand_probs[my_hand][dealer_hand]
    if dealer_hand > 21:
        stand_probs[my_hand][dealer_hand] = 1
        return 1
    if my_hand > 21 or dealer_hand >= my_hand:
        stand_probs[my_hand][dealer_hand] = 0
        return 0
    probs = []
    for card in cards:
        if card == 1:
            prob1 = stand_prob(my_hand,dealer_hand+1)
            prob11 = stand_prob(my_hand, dealer_hand+11)
            probs.append(min(prob1, prob11))
        else:
            probs.append(stand_prob(my_hand, dealer_hand+card))
    prob = sum(probs)/len(probs)
    stand_probs[my_hand][dealer_hand] = prob
    return prob

stand_prob(21, 2)


0.8379522789278752

In [179]:
# stand_probs.style.background_gradient(cmap='viridis')
def color_background(val):
    color = 'darkred' if val < .25 else 'red' if val < .5 else 'green' if val < .75 else 'darkgreen' if val < 1 else 0
    
    return 'color: %s' % color

stand_probs.loc[:21,:21].style.applymap(color_background)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
8,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
9,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
10,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.148438,nan,nan,nan,nan,nan


In [180]:
columns = range(1,33)
index = range(1,33)
hit_probs = pd.DataFrame(columns=columns, index=index)
def hit_prob(my_hand, dealer_hand):
    if not pd.isna(hit_probs[my_hand][dealer_hand]):
        return hit_probs[my_hand][dealer_hand]
    if my_hand > 21:
        hit_probs[my_hand][dealer_hand] = 0
        return 0
    probs = []
    for card in cards:
        if card == 1:
            s_prob1 = stand_prob(my_hand+1, dealer_hand)
            h_prob1 = hit_prob(my_hand+1, dealer_hand)
            s_prob11 = stand_prob(my_hand+11, dealer_hand)
            h_prob11 = hit_prob(my_hand+11, dealer_hand)
            probs.append(max(s_prob1, h_prob1, s_prob11, h_prob11))
        else:
            s_prob = stand_prob(my_hand+card, dealer_hand)
            h_prob = hit_prob(my_hand+card, dealer_hand)
            probs.append(max(s_prob, h_prob))
    prob = sum(probs)/len(probs)
    hit_probs[my_hand][dealer_hand] = prob
    return prob

hit_prob(16, 6)

0.24269069317837771

In [214]:
s = hit_probs.loc[:11,:21].style.applymap(color_background)
s


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
1,nan,0.32473,0.315286,0.305534,0.295566,0.287586,0.309215,0.341126,0.382469,0.436331,0.470348,0.269508,0.250258,0.232382,0.215783,0.199799,0.177386,0.147428,0.108733,0.0600359,0
2,nan,0.372532,0.361628,0.35038,0.338894,0.342494,0.365606,0.397509,0.438531,0.491765,0.524864,0.307813,0.285826,0.26541,0.246452,0.224747,0.196824,0.161591,0.117883,0.0644579,0
3,nan,0.382683,0.371457,0.359883,0.348068,0.356118,0.37904,0.410271,0.45034,0.502296,0.534435,0.315741,0.293188,0.272246,0.2528,0.229414,0.20002,0.163559,0.118891,0.0648021,0
4,nan,0.393154,0.381598,0.369686,0.357529,0.370319,0.392856,0.423411,0.462512,0.513158,0.544306,0.323908,0.300772,0.279288,0.259339,0.234229,0.203326,0.165598,0.119939,0.0651607,0
5,nan,0.402018,0.390184,0.37799,0.365547,0.379602,0.40513,0.43513,0.473368,0.522796,0.55293,0.330888,0.307253,0.285306,0.264927,0.239148,0.206709,0.16769,0.121015,0.0655299,0
6,nan,0.395496,0.383901,0.371944,0.359737,0.349182,0.39787,0.432671,0.470998,0.520155,0.549261,0.326431,0.303115,0.281464,0.261359,0.242691,0.210141,0.169815,0.122111,0.0659064,0
7,nan,0.37623,0.365273,0.353962,0.342404,0.33069,0.339827,0.412294,0.457047,0.506762,0.534833,0.311973,0.28969,0.268997,0.249783,0.231942,0.211764,0.171958,0.123217,0.0662872,0
8,nan,0.348935,0.338858,0.32844,0.317781,0.30697,0.308609,0.342978,0.428767,0.486357,0.513393,0.290999,0.270213,0.250912,0.23299,0.216348,0.197522,0.171849,0.124328,0.0666699,0
9,nan,0.314572,0.305544,0.296202,0.286635,0.276924,0.280908,0.304856,0.348805,0.450794,0.484891,0.263468,0.244649,0.227174,0.210947,0.195879,0.178308,0.154347,0.122739,0.0670521,0
10,nan,0.272598,0.264773,0.256675,0.248383,0.239966,0.248288,0.272091,0.30353,0.359107,0.43791,0.228264,0.21196,0.19682,0.182761,0.169707,0.153391,0.131141,0.10179,0.064029,0


In [182]:
def best_move(my_hand, dealer_hand):
    return 'h' if hit_prob(my_hand, dealer_hand)>stand_prob(my_hand, dealer_hand) else 's'

table = []
index = range(2,21)
columns = range(1,12)
for i in index:
    table.append([best_move(i, j) for j in columns])

pd.DataFrame(table, index=index, columns=columns)

,1,2,3,4,5,6,7,8,9,10,11
2,h,h,h,h,h,h,h,h,h,h,h
3,h,h,h,h,h,h,h,h,h,h,h
4,h,h,h,h,h,h,h,h,h,h,h
5,h,h,h,h,h,h,h,h,h,h,h
6,h,h,h,h,h,h,h,h,h,h,h
7,h,h,h,h,h,h,h,h,h,h,h
8,h,h,h,h,h,h,h,h,h,h,h
9,h,h,h,h,h,h,h,h,h,h,h
10,h,h,h,h,h,h,h,h,h,h,h
11,h,h,h,h,h,h,h,h,h,h,h


In [166]:
best_probs = pd.concat([stand_probs, hit_probs]).max(level=0)

In [183]:
best_probs.loc[:21,:21].style.applymap(color_background)



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
1,nan,0.417052,0.404896,0.392349,0.379529,0.376369,0.402812,0.440199,0.487756,0.552313,0.602974,0.345619,0.320932,0.298008,0.276722,0.291365,0.389456,0.503034,0.633063,0.780467,1
2,nan,0.456775,0.443372,0.429553,0.415446,0.425196,0.453097,0.490556,0.538077,0.599136,0.636531,0.376779,0.349866,0.324876,0.30167,0.363008,0.458028,0.568203,0.694523,0.837952,1
3,nan,0.464364,0.450716,0.436649,0.422291,0.43639,0.463769,0.500412,0.546829,0.606445,0.642808,0.382607,0.355278,0.329901,0.306337,0.382113,0.474001,0.580679,0.703157,0.842427,1
4,nan,0.474102,0.460134,0.445744,0.435114,0.449859,0.476586,0.512279,0.557452,0.615465,0.650752,0.389984,0.362128,0.336262,0.326428,0.401739,0.490457,0.593576,0.712115,0.847089,1
5,nan,0.482571,0.46833,0.453661,0.443753,0.462089,0.488264,0.523094,0.56709,0.623553,0.657732,0.396465,0.368146,0.34185,0.335127,0.421714,0.507246,0.606769,0.721309,0.851889,1
6,nan,0.485873,0.471535,0.456766,0.441701,0.464653,0.49497,0.529278,0.572407,0.627616,0.660666,0.39919,0.370676,0.344199,0.319614,0.42315,0.524236,0.620152,0.730657,0.856783,1
7,nan,0.470958,0.45713,0.442874,0.428322,0.416405,0.478053,0.518561,0.561953,0.617029,0.648937,0.388299,0.360563,0.334808,0.310894,0.288687,0.517474,0.633629,0.740093,0.861734,1
8,nan,0.445655,0.432656,0.41924,0.405534,0.391647,0.409288,0.490803,0.54192,0.597547,0.628307,0.369142,0.342775,0.318291,0.295556,0.274445,0.333749,0.617768,0.749554,0.866709,1
9,nan,0.412168,0.400243,0.387918,0.375312,0.362528,0.370958,0.410658,0.506034,0.570863,0.600476,0.343299,0.318778,0.296008,0.274864,0.255231,0.311499,0.410903,0.723926,0.871678,1
10,nan,0.370523,0.359863,0.348835,0.337546,0.32609,0.337301,0.364564,0.413388,0.5257,0.564382,0.309784,0.287656,0.267109,0.24803,0.230314,0.289249,0.381553,0.490898,0.832377,1


In [190]:

hit_probs_plus = hit_probs+sys.float_info.epsilon

In [203]:
(stand_probs/hit_probs_plus).loc[:11,13:21].T

,1,2,3,4,5,6,7,8,9,10,11
13,0.139077,0.393248,0.131733,0.119813,0.109431,0.103497,0.101042,0.101068,0.104187,0.111666,0.1236
14,0.34617,0.59969,0.658859,0.302722,0.276492,0.261498,0.255294,0.255362,0.263243,0.282138,0.312291
15,0.638619,0.870461,0.944503,0.978433,0.518546,0.490426,0.478791,0.478918,0.493698,0.529135,0.585686
16,1.04004,1.25546,1.32523,1.39363,1.40134,0.810687,0.791455,0.791664,0.816096,0.874674,0.968154
17,1.64255,1.84433,1.91037,1.97584,2.04014,2.01365,1.2387,1.23906,1.28107,1.3828,1.54819
18,2.64167,2.8345,2.89805,2.96172,3.02491,3.0871,3.0093,1.94211,2.01818,2.20564,2.52059
19,4.62632,4.82007,4.88413,4.94899,5.01399,5.0786,5.14236,4.96886,3.34779,3.74842,4.4676
20,10.5447,10.7748,10.8508,10.9286,11.0073,11.0863,11.1649,11.2428,10.7965,7.66681,10.3037
21,3.51491e+15,3.7738e+15,3.79395e+15,3.81495e+15,3.83657e+15,3.85861e+15,3.88091e+15,3.90331e+15,3.92569e+15,3.74869e+15,2.78935e+15
